# MiniProject 3: Decision Tree
### How to run this notebook [if you need to*]
- **Run all cells in order using _"Run All"_ in the Cell menu**
- If you wish to re-run a cell, you must re-run all cells in order after restarting the kernel

- In certain cases, if the output of a cell is too large, you can click the _"Open full output data in text editor"_ button to view it. 

- *If any cell returns an error, pressing restart and re-running all cells in order should fix it.*

*The second dataset can take 7-10 minutes to run.

### Introduction
In computer science, decision tree learning uses a decision tree (as a predictive model) to go
from observations about an item (represented in the branches) to conclusions about the item's
target value (represented in the leaves). It is one of the predictive modeling approaches used
in statistics, data mining and machine learning. Tree models where the target variable can
take a discrete set of values are called classification trees; in these tree structures, leaves
represent class labels and branches represent conjunctions of features that lead to those class
labels. Decision trees where the target variable can take continuous values (typically real
numbers) are called regression trees. In decision analysis, a decision tree can be used to
visually and explicitly represent decisions and decision making. In data mining, a decision tree
describes data (but the resulting classification tree can be an input for decision making). This
page deals with decision trees in data mining.

*Instead of growing full trees, you will use an early stopping strategy. To this end, we
will impose a limit on the minimum number of instances at a leaf node, let this
threshold be denoted as nmin, where nmin is described as a percentage relative to the
size of the training dataset. For example, if the size of the training dataset is 150
and nmin= 5, then a node will only be split further if it has more than eight instances*

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


### Recursive function for building the Decision Tree

In [20]:
def build_tree(X, y, min_leaf=2, depth=0):
    num_samples, num_features = X.shape
    num_labels = len(np.unique(y))

    # Base case for recursive function build_tree
    if num_labels == 1 or num_samples < min_leaf:
        most_common_label = np.argmax(np.bincount(y))
        return {'feature': None, 'threshold': None, 'left': None, 'right': None, 'value': most_common_label}

    # finding the best split for the current node
    best_score = -1
    for feat in range(num_features):
        X_feat = X[:, feat]
        thresholds = np.unique(X_feat)
        for thresh in thresholds:
            score = information_gain(X_feat, y, thresh)

            if score > best_score:
                best_score = score
                best_feat = feat
                best_thresh = thresh

    # grow children recursively
    left_idx = np.where(X[:, best_feat] <= best_thresh)[0]
    right_idx = np.where(X[:, best_feat] > best_thresh)[0]
    
    left_child = build_tree(X[left_idx, :], y[left_idx], min_leaf, depth+1)
    right_child = build_tree(X[right_idx, :], y[right_idx], min_leaf, depth+1)

    return {'feature': best_feat, 'threshold': best_thresh, 'left': left_child, 'right': right_child, 'value': None}

def entropy(y):
    proportions = np.bincount(y) / len(y)
    entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
    return entropy


def information_gain(X, y, thresh):
    parent_loss = entropy(y)
    left_idx = np.where(X <= thresh)[0]
    right_idx = np.where(X > thresh)[0]
    n, n_left, n_right = len(y), len(left_idx), len(right_idx)

    if n_left == 0 or n_right == 0:
        return 0

    child_loss = (n_left / n) * entropy(y[left_idx]) + (n_right / n) * entropy(y[right_idx])
    return parent_loss - child_loss


In [11]:
# function to traverse the tree recursively
def traverse_tree(x, node):
    if node['value'] is not None:
        return node['value']

    if x[node['feature']] <= node['threshold']:
        return traverse_tree(x, node['left'])
    return traverse_tree(x, node['right'])

# function to make predictions on a test set
def predict(X, tree):
    predictions = [traverse_tree(x, tree) for x in X]
    return np.array(predictions)

# traverse the tree and print the tree
def print_tree(node, label, depth=0):

    if node['value'] is not None:
        print('\t' * depth, 'Predict', label[node['value']])
        return

    print('\t' * depth, 'X[{}] <= {}'.format(node['feature'], node['threshold']))
    print_tree(node['left'], label, depth + 1)
    print('\t' * depth, 'X[{}] > {}'.format(node['feature'], node['threshold']))
    print_tree(node['right'], label, depth + 1)


## Dataset 1: Iris
Iris: has three classes and the task is to accurately predict one of the three subtypes
of the Iris flower given four different physical features. These features include the
length and width of the sepals and the petals. There is a total of 150 instances with
each class having 50 instances.

*For the Iris dataset use nmin E {5, 10, 15, 20}, and calculate the accuracy using
10 fold cross-validation for each value of min.*


In [18]:
df = pd.read_csv('iris.csv', header=None)
data = df.to_numpy()

# change species to numerical values
for i in range(len(data)):
    if data[i][4] == 'Iris-setosa':
        data[i][4] = int(0)
    elif data[i][4] == 'Iris-versicolor':
        data[i][4] = int(1)
    elif data[i][4] == 'Iris-virginica':
        data[i][4] = int(2)

X = data[:, :-1]
y = data[:, -1]
 
# fix dtype(0) error
X = X.astype(np.float64)
y = y.astype(np.float64)
label_to_int = {label: i for i, label in enumerate(np.unique(y))}
y = np.array([label_to_int[label] for label in y])

kf = KFold(n_splits=10, shuffle=True, random_state=8966)

nmin = [0.05, 0.10, 0.15, 0.20]
for i in nmin:
    clf_list = []
    avg_accuracy = []
    std_dev = []
    # iterate over each fold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        iris_clf = build_tree(X_train, y_train, i*len(X_train))
        y_pred = predict(X_test, iris_clf)

        clf_list.append(iris_clf)
        avg_accuracy.append(accuracy_score(y_test, y_pred))
        std_dev.append(np.std(y_pred == y_test) / np.sqrt(y_pred.shape[0]))
    print('---------------------------------------------')
    print('Average accuracy for N_min =', i, 'is', np.mean(avg_accuracy))
    print('Standard deviation for N_min =', i, 'is', np.mean(std_dev))

print('---------------------------------------------')
print_tree(clf_list[0], ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])



---------------------------------------------
Average accuracy for N_min = 0.05 is 0.9400000000000001
Standard deviation for N_min = 0.05 is 0.04975720846542675
---------------------------------------------
Average accuracy for N_min = 0.1 is 0.9266666666666667
Standard deviation for N_min = 0.1 is 0.058534282980151855
---------------------------------------------
Average accuracy for N_min = 0.15 is 0.9266666666666667
Standard deviation for N_min = 0.15 is 0.058534282980151855
---------------------------------------------
Average accuracy for N_min = 0.2 is 0.9266666666666667
Standard deviation for N_min = 0.2 is 0.058534282980151855
---------------------------------------------
 X[2] <= 1.9
	 Predict Iris-setosa
 X[2] > 1.9
	 X[3] <= 1.7
		 X[2] <= 5.0
			 X[0] <= 4.9
				 Predict Iris-versicolor
			 X[0] > 4.9
				 Predict Iris-versicolor
		 X[2] > 5.0
			 Predict Iris-virginica
	 X[3] > 1.7
		 X[2] <= 4.8
			 Predict Iris-virginica
		 X[2] > 4.8
			 Predict Iris-virginica


## Dataset 2: Spambase

Spambase: is a binary classification task and the objective is to classify email
messages as being spam or not. To this end the dataset uses for seven text-based
features to represent each email message. There are about 4600 instances.

*For the Spambase dataset use nmin E {5, 10, 15, 20, 25}, and calculate the
accuracy using 10 fold cross-validation for each value of nmin.*

In [19]:
data = np.genfromtxt('spambase.csv', delimiter=',')
X = data[:, :-1]
y = data[:, -1]

label_to_int = {label: i for i, label in enumerate(np.unique(y))}
y = np.array([label_to_int[label] for label in y])

kf = KFold(n_splits=10, shuffle=True, random_state=101)

nmin = [0.05, 0.10, 0.15, 0.20, 0.25]
for i in nmin:
    avg_accuracy = []
    std_dev = []
    spam_clf_list = []
    # iterate over each fold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        spam_clf = build_tree(X_train, y_train, i*len(X_train))
        y_pred = predict(X_test, spam_clf)

        spam_clf_list.append(spam_clf)
        avg_accuracy.append(accuracy_score(y_test, y_pred))
        std_dev.append(np.std(y_pred == y_test) / np.sqrt(y_pred.shape[0]))
    print('---------------------------------------------')
    print('Average accuracy for N_min =', i, 'is', np.mean(avg_accuracy))
    print('Standard deviation for N_min =', i, 'is', np.mean(std_dev))


---------------------------------------------
Average accuracy for N_min = 0.05 is 0.9015495614448741
Standard deviation for N_min = 0.05 is 0.013755602043015975
---------------------------------------------
Average accuracy for N_min = 0.1 is 0.8958936150146185
Standard deviation for N_min = 0.1 is 0.014194888988927368
---------------------------------------------
Average accuracy for N_min = 0.15 is 0.8765462604923135
Standard deviation for N_min = 0.15 is 0.015281801132000988
---------------------------------------------
Average accuracy for N_min = 0.2 is 0.8628586249174761
Standard deviation for N_min = 0.2 is 0.016000284751655894
---------------------------------------------
Average accuracy for N_min = 0.25 is 0.8300396114307272
Standard deviation for N_min = 0.25 is 0.017433830691136826



__In both the datasets, we can see that the smaller the value of N_min, the better the accuracy. This is because the smaller the value of N_min, the more the number of splits and hence the more the number of nodes. This leads to a more complex tree and hence a better accuracy.__

__But, this can also lead to overfitting. Hence, we need to find the optimal value of N_min which gives the best accuracy without overfitting.__
